In [1]:
import pandas as pd
import numpy as np
import re
import math
from random import sample, choice
from sklearn.model_selection import train_test_split
import warnings
import os
warnings.filterwarnings('ignore')
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import KFold
from tqdm import tqdm

from matplotlib import pyplot as plt

This includes kFold cross validation (10) and it grabs the max(accuracy)

In [2]:
#### only needs to be run once ####
def multiple_max(df):
    df1 = df.drop('dataset', axis=1)

    maxes = {}      # {dataset: [max1, max2, ...]}
    max_val = {}    # {dataset: max_val}
    for i in range(df.shape[0]):
        row = np.array(df1.iloc[i])
        row_maxes = np.argwhere(row == np.amax(row)).flatten().tolist()
        max_val[df.iloc[i, 0]] = np.amax(row)
        maxes[df.iloc[i, 0]] = row_maxes
        # maxes[df.iloc[i, 0]] = df1.columns[row_maxes].tolist() # if you want col names
    return maxes, max_val

pre = pd.read_csv("emp_results.csv")
pre['dataset'] = pre['dataset'].apply(lambda x: int(re.split(r'\.arff', x)[0]))

optimized_cols = []
default_cols = []

for col in pre.columns:
    if col == 'dataset':
        optimized_cols.append(col)
        default_cols.append(col)
    if "+" in col:
        optimized_cols.append(col)
    elif "-" in col:
        default_cols.append(col)

optim_maxes, optim_max_values = multiple_max(pre[optimized_cols])
default_maxes, default_max_values = multiple_max(pre[default_cols])

# optim_maxes = {dataset: [max1, max2, ..., max466]}
# default_maxes = {dataset: [max1, max2, ..., max466]}

In [3]:
# redo train_meta_model but with 2 fold cross validation
def train_meta_model(sk_algorithm, X, y, valid_datasets, maxes):
	accuracies = []
	# for i in range(iters):
	# proably want to shuffle the data before running kfold 
	# to do this, need to combine X, y, and valid_datasets into one dataframe
	# I think we want to combine valid_dataset, becuase it is the key? 
	kf = KFold(n_splits=10)
	kf.get_n_splits(X)
	for train_index, test_index in kf.split(X):
		X_train, X_test = X[train_index], X[test_index]
		y_train, y_test = y[train_index], y[test_index] # dont use y_test
		ds_train, ds_test = valid_datasets[train_index], valid_datasets[test_index] # valid_datasets is the list of datasets
		model = sk_algorithm
		model.fit(X_train, y_train)
		y_pred = model.predict(X_test)
		correct = 0
		for ds, prediction in zip(ds_test, y_pred):
			if prediction in maxes[ds]:
				correct += 1
		accuracies.append(correct/len(ds_test))
	return max(accuracies)

In [4]:
# Variables for meta-model
models = ["Metadataset", "RandomForest", "LogisticRegression", "SVC", "KNeighbors",
          "GaussianNB", "DecisionTree", "AdaBoost", "GradientBoosting",
          "MLP", "Bagging", "ExtraTrees",  "Voting"]
results = []
maxes = optim_maxes #optim_maxes_sd
iters = 5 # not using this anymore 

meta_datasets_path = "meta_datasets"
meta_datasets = {}
for meta_dataset in os.listdir(meta_datasets_path):
    meta_datasets[meta_dataset] = pd.read_csv(os.path.join(meta_datasets_path, meta_dataset))

In [5]:
for df_name, df in tqdm(meta_datasets.items(), desc="Meta-datasets"):
    # df_name, df = choice(list(meta_datasets.items()))
    print('df_name:', df_name)
    results_arr = [df_name]
    X = []
    y = []
    valid_datasets = []

    # df_name, df = choice(list(meta_datasets.items()))
    df.replace(np.nan, 0, inplace=True)
    df.replace(np.inf, 0, inplace=True)
    
    for i in range(df.shape[0]):
        if df.loc[i, 'dataset'] in maxes:
            y.append(sample(maxes[df.loc[i, 'dataset']],k=1)[0])
            X.append(df.iloc[i, 1:].tolist())
            valid_datasets.append(df.loc[i, 'dataset'])
    print(len(valid_datasets))
    X, y, valid_datasets = np.array(X), np.array(y), np.array(valid_datasets)

    # Random Forest
    rf = RandomForestClassifier(n_estimators=100, max_depth=2, random_state=0)
    rf_accuracies = train_meta_model(rf, X, y, valid_datasets, maxes)
    results_arr.append(np.mean(rf_accuracies))
    # print(f'rf accuracy: {round(np.mean(rf_accuracies)*100,2)}%', flush=True)

    # Logistic Regression
    lr = LogisticRegression(random_state=0, solver='lbfgs', multi_class='multinomial')
    lr_accuracies = train_meta_model(lr, X, y, valid_datasets, maxes)
    results_arr.append(np.mean(lr_accuracies))
    # print(f'lr accuracy: {round(np.mean(lr_accuracies)*100,2)}%', flush=True)

    # SVM
    svm = SVC(gamma='auto')
    svm_accuracies = train_meta_model(svm, X, y, valid_datasets, maxes)
    results_arr.append(np.mean(svm_accuracies))
    # print(f'svm accuracy: {round(np.mean(svm_accuracies)*100,2)}%', flush=True)

    # KNN
    knn = KNeighborsClassifier(n_neighbors=3)
    knn_accuracies = train_meta_model(knn, X, y, valid_datasets, maxes)
    results_arr.append(np.mean(knn_accuracies))
    # print(f'knn accuracy: {round(np.mean(knn_accuracies)*100,2)}%', flush=True)

    # Naive Bayes
    nb = GaussianNB()
    nb_accuracies = train_meta_model(nb, X, y, valid_datasets, maxes)
    results_arr.append(np.mean(nb_accuracies))
    # print(f'nb accuracy: {round(np.mean(nb_accuracies)*100,2)}%')

    # Decision Tree
    dt = DecisionTreeClassifier(random_state=0)
    dt_accuracies = train_meta_model(dt, X, y, valid_datasets, maxes)
    results_arr.append(np.mean(dt_accuracies))
    # print(f'dt accuracy: {round(np.mean(dt_accuracies)*100,2)}%')

    # AdaBoost
    ab = AdaBoostClassifier(n_estimators=100, random_state=0)
    ab_accuracies = train_meta_model(ab, X, y, valid_datasets, maxes)
    results_arr.append(np.mean(ab_accuracies))
    # print(f'ab accuracy: {round(np.mean(ab_accuracies)*100,2)}%')

    # Gradient Boosting
    gb = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0)
    gb_accuracies = train_meta_model(gb, X, y, valid_datasets, maxes)
    results_arr.append(np.mean(gb_accuracies))
    # print(f'gb accuracy: {round(np.mean(gb_accuracies)*100,2)}%')

    # Neural Network
    nn = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1)
    nn_accuracies = train_meta_model(nn, X, y, valid_datasets, maxes)
    results_arr.append(np.mean(nn_accuracies))
    # print(f'nn accuracy: {round(np.mean(nn_accuracies)*100,2)}%')

    # Bagging
    bg = BaggingClassifier(base_estimator=DecisionTreeClassifier(), n_estimators=10, random_state=0)
    bg_accuracies = train_meta_model(bg, X, y, valid_datasets, maxes)
    results_arr.append(np.mean(bg_accuracies))
    # print(f'bg accuracy: {round(np.mean(bg_accuracies)*100,2)}%')

    # Extra Trees
    et = ExtraTreesClassifier(n_estimators=100, max_depth=None, min_samples_split=2, random_state=0)
    et_accuracies = train_meta_model(et, X, y, valid_datasets, maxes)
    results_arr.append(np.mean(et_accuracies))
    # print(f'et accuracy: {round(np.mean(et_accuracies)*100,2)}%')

    # Voting Classifier
    vc = VotingClassifier(estimators=[('rf', rf), ('lr', lr), ('svm', svm), ('knn', knn), ('nb', nb), ('dt', dt), ('ab', ab), ('gb', gb), ('nn', nn), ('bg', bg), ('et', et)], voting='hard')
    vc_accuracies = train_meta_model(vc, X, y, valid_datasets, maxes)
    results_arr.append(np.mean(vc_accuracies))
    # print(f'vc accuracy: {round(np.mean(vc_accuracies)*100,2)}%')

    results.append(results_arr)

Meta-datasets:   0%|          | 0/13 [00:00<?, ?it/s]

df_name: concept_metadataset.csv
177


Meta-datasets:   8%|▊         | 1/13 [00:17<03:35, 17.97s/it]

df_name: clustering_metadataset.csv
428


Meta-datasets:  15%|█▌        | 2/13 [00:41<03:55, 21.43s/it]

...

Meta-datasets:  85%|████████▍ | 11/13 [07:23<01:38, 49.35s/it]

df_name: statistical_metadataset.csv
437


Meta-datasets:  92%|█████████▏| 12/13 [08:09<00:48, 48.41s/it]

df_name: relative_metadataset.csv
434


Meta-datasets: 100%|██████████| 13/13 [08:30<00:00, 39.26s/it]


In [6]:
models = ["Metadataset", "RandomForest", "LogisticRegression", "SVC", "KNeighbors",
          "GaussianNB", "DecisionTree", "AdaBoost", "GradientBoosting",
          "MLP", "Bagging", "ExtraTrees",  "Voting"]

df = pd.DataFrame(results, columns=models)
df

,Metadataset,RandomForest,LogisticRegression,SVC,KNeighbors,GaussianNB,DecisionTree,AdaBoost,GradientBoosting,MLP,Bagging,ExtraTrees,Voting
0,concept_metadataset.csv,0.555556,0.611111,0.555556,0.555556,0.444444,0.444444,0.500000,0.352941,0.500000,0.500000,0.444444,0.555556
1,clustering_metadataset.csv,0.511628,0.558140,0.604651,0.441860,0.325581,0.441860,0.465116,0.627907,0.511628,0.511628,0.651163,0.627907
2,all_metadataset.csv,0.775000,0.450000,0.575000,0.400000,0.400000,0.475000,0.575000,0.425000,0.275000,0.650000,0.725000,0.750000
3,general_metadataset.csv,0.704545,0.590909,0.568182,0.454545,0.318182,0.590909,0.386364,0.590909,0.545455,0.590909,0.568182,0.636364
4,model-based_metadataset.csv,0.525000,0.358974,0.625000,0.435897,0.435897,0.512821,0.461538,0.525000,0.307692,0.461538,0.487179,0.512821
5,landmarking_metadataset.csv,0.581395,0.534884,0.534884,0.651163,0.511628,0.604651,0.488372,0.674419,0.500000,0.581395,0.651163,0.604651
6,itemset_metadataset.csv,0.522727,0.522727,0.545455,0.395349,0.522727,0.500000,0.418605,0.454545,0.545455,0.454545,0.454545,0.545455
7,complexity_metadataset.csv,0.725000,0.350000,0.575000,0.550000,0.275000,0.650000,0.650000,0.600000,0.575000,0.675000,0.725000,0.775000
8,info-theory_metadataset.csv,0.545455,0.545455,0.568182,0.348837,0.363636,0.454545,0.522727,0.386364,0.545455,0.522727,0.590909,0.568182
9,d2v_pretrained_metadaset.csv,0.522727,0.568182,0.545455,0.431818,0.295455,0.431818,0.386364,0.477273,0.227273,0.386364,0.395349,0.613636


In [ ]:
models = ["Metadataset", "RandomForest", "LogisticRegression", "SVC", "KNeighbors",
          "GaussianNB", "DecisionTree", "AdaBoost", "GradientBoosting",
          "MLP", "Bagging", "ExtraTrees",  "Voting"]

df = pd.DataFrame(results, columns=models)
df

In [7]:
# df.to_csv('results.csv')
# df.to_csv('results_sd.csv', index=False)
# df.to_csv('results.csv', index=False)
# df.to_csv('results_sd_2.csv', index=False)
# df.to_csv('results_cross_fold.csv', index=False)

df.to_csv('results_some_changes.csv', index=False)